In [1]:
import os

In [2]:
%pwd

'C:\\Users\\Franklin NDJATHE\\Documents\\projects\\End-to-End-ML-Project-with-MLFlow\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'C:\\Users\\Franklin NDJATHE\\Documents\\projects\\End-to-End-ML-Project-with-MLFlow'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [10]:
from typing import ClassVar
from box import ConfigBox

@dataclass
class ConfigurationManager:
    config_filepath: Path = CONFIG_FILE_PATH
    params_filepath: Path = PARAMS_FILE_PATH
    schema_filepath: Path = SCHEMA_FILE_PATH
    
    config: ClassVar[ConfigBox] = read_yaml(config_filepath)
    params: ClassVar[ConfigBox] = read_yaml(params_filepath)
    schema: ClassVar[ConfigBox] = read_yaml(schema_filepath)
    
    def __post_init__(self):
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

[2023-07-29 13:50:22,848: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-29 13:50:22,849: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-29 13:50:22,851: INFO: common: yaml file: schema.yaml loaded successfully]


In [11]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import  get_size

In [12]:
@dataclass
class DataIngestion:
    config: DataIngestionConfig
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url= self.config.source_url,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        :return: None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
    

[2023-07-29 14:21:34,987: INFO: common: created directory at: artifacts]
[2023-07-29 14:21:34,991: INFO: common: created directory at: artifacts/data_ingestion]
[2023-07-29 14:21:35,674: INFO: 401826441: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 659C:4A0B:9E9249:B4E525:64C4FFE1
Accept-Ranges: bytes
Date: Sat, 29 Jul 2023 12:21:41 GMT
Via: 1.1 varnish
X-Served-By: cache-mrs10536-MRS
X-Cache: HIT
X-Cache-Hits: 1
X-Timer: S1690633301.297650,VS0,VE167
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fast